In [1]:
import socket
import threading
import cv2
import numpy
s=socket.socket()
s.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
ip=""
port=2222
s.bind((ip,port))
s.listen()
csession,addr=s.accept()
cap=cv2.VideoCapture(0)
def sender():
    while True:
        ret,photo=cap.read()
        photo = cv2.resize(photo, (500,500))
        if ret:
            csession.send(numpy.ndarray.tobytes(photo))
        else:
            print("Could not Send")
def recv():
    while True:
        data=csession.recv(10000000000)
        if(data == b'finished'): 
            print("Finished")
            cap.release()
            cv2.destroyAllWindows()
            csession.close()
        photo=numpy.frombuffer(data,dtype=numpy.uint8)
        #print(len(photo))        
        if len(photo)==500*500*3:
            cv2.imshow('From Server',photo.reshape(500,500,3))
            if cv2.waitKey(1)==13:
                csession.send(b"finished")
                cap.release()
                cv2.destroyAllWindows()
                break
t1=threading.Thread(target=sender)
t1.start()
t2=threading.Thread(target=recv)
t2.start()        
            
    